In [3]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import Q1500US
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import SimpleMovingAverage,AverageDollarVolume

In [4]:
universe = Q1500US()

In [5]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [6]:
#Get technology sector
technology_sector = sector.eq(311)

In [7]:
dollar_volume = AverageDollarVolume(window_length = 30) 

In [8]:
#Top 5 securities based off average dollar value
#Top is top 10 percent
#Bottom is bottom 10 percent
#Percentile between is any other range

In [9]:
high_dollar_volume = dollar_volume.percentile_between(90,100)

In [10]:
top_open_prices = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

In [ ]:
high_close_price = USEquityPricing.close.latest.percentile_between(90,100,mask=top_open_prices)

In [12]:
def make_pipline():
    
    #Universe Q1500US
    base_universe = Q1500US()
    
    #Technology Sector
    technology_sector = sector.eq(309)
    
    #Make Mask of 1500US and Technology Sector
    base_technology = base_universe & technology_sector
    
    #Dollar Volume (30 days) 
    dollar_volume = AverageDollarVolume(window_length=30)
    
    #Get top 5% in Average Dollar Volume
    top_dollar_volume = dollar_volume.percentile_between(95,100)
    
    #Combine Filters 
    top_five_base_technology = base_technology & top_dollar_volume
    
    #10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], 
                                  window_length=10,
                                  mask=top_five_base_technology)
    
    #30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], 
                                  window_length=30,
                                  mask=top_five_base_technology)
    
    #Percentile Difference
    percent_difference = (mean_10 - mean_30) / mean_30
    
    #List of stocks to short
    shorts = percent_difference < 0
    
    #List of stocks to long
    longs = percent_difference > 0
    
    #Final filter and mask for shorts and longs
    final_securities = (shorts | longs)
    
    #Return the pipeline
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'percent_difference':percent_difference
    },screen=final_securities)
    
    
    

In [15]:
results = run_pipeline(make_pipline(),'2015-05-05','2015-05-05')

In [16]:
results

longs  percent_difference  \
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True            0.036528   
                          Equity(448 [APA])     True            0.035926   
                          Equity(455 [APC])     True            0.049153   
                          Equity(858 [BHI])     True            0.033807   
                          Equity(1746 [COG])    True            0.058078   
                          Equity(2368 [DVN])    True            0.046264   
                          Equity(2564 [EOG])    True            0.032102   
                          Equity(2621 [VAL])    True            0.060197   
                          Equity(3443 [HAL])    True            0.049257   
                          Equity(3647 [HP])     True            0.040991   
                          Equity(5035 [MRO])    True            0.061598   
                          Equity(5213 [NBL])    True            0.010443   
                          Equity(5214 [NBR])    True            0.064133   
                          Equity(5249 [NE])     True            0.037559   
                          Equity(5729 [OXY])    True            0.029776   
                          Equity(6928 [SLB])    True            0.046555   
                          Equity(7244 [SWN])    True            0.070788   
                          Equity(7612 [ANDV])   True            0.005997   
                          Equity(7990 [VLO])   False           -0.017145   
                          Equity(8214 [WMB])    True            0.018876   
                          Equity(8347 [XOM])    True            0.017343   
                          Equity(8461 [CHK])    True            0.014265   
                          Equity(9038 [RIG])    True            0.048180   
                          Equity(13176 [CAM])   True            0.082110   
                          Equity(17436 [PXD])   True            0.010248   
                          Equity(19249 [RRC])   True            0.087062   
                          Equity(19336 [WFT])   True            0.049141   
                          Equity(22784 [FTI])   True            0.054529   
                          Equity(23112 [CVX])   True            0.018972   
                          Equity(23998 [COP])   True            0.023902   
                          Equity(24809 [NOV])   True            0.024940   
                          Equity(25707 [WLL])   True            0.048205   
                          Equity(33856 [CLR])   True            0.064304   
                          Equity(34440 [CXO])   True            0.042184   
                          Equity(39797 [OAS])   True            0.042388   
                          Equity(40852 [KMI])   True            0.023016   
                          Equity(41636 [MPC])   True            0.011952   
                          Equity(42788 [PSX])   True            0.020911   

                                              shorts  
2015-05-05 00:00:00+00:00 Equity(216 [HES])    False  
                          Equity(448 [APA])    False  
                          Equity(455 [APC])    False  
                          Equity(858 [BHI])    False  
                          Equity(1746 [COG])   False  
                          Equity(2368 [DVN])   False  
                          Equity(2564 [EOG])   False  
                          Equity(2621 [VAL])   False  
                          Equity(3443 [HAL])   False  
                          Equity(3647 [HP])    False  
                          Equity(5035 [MRO])   False  
                          Equity(5213 [NBL])   False  
                          Equity(5214 [NBR])   False  
                          Equity(5249 [NE])    False  
                          Equity(5729 [OXY])   False  
                          Equity(6928 [SLB])   False  
                          Equity(7244 [SWN])   False  
                          Equity(7612 [ANDV])  False  
                          Equity(7990

# Pipeline - Technology Sector Algorithm 
---
*Note: Ran on Quantopian's Backtesting Platform*

In [ ]:
from quantopian.algorithm import attach_pipeline,pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar


def initialize(context):
    
    schedule_function(my_rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(hours=1))
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')


    
    
    
def my_rebalance(context,data):
    for security in context.portfolio.positions:
        #Exit out of position if security is not in our long or short list
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
            
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)


def my_compute_weights(context):
    
    #Set to zero to avoid division by zero error
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
  
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = -0.5 / len(context.shorts)
    
    return (long_weight,short_weight)




def before_trading_start(context,data):
    
    #Grab pipeline object
    context.output = pipeline_output('my_pipeline')
    
    #Securties to go long on
    context.longs = context.output[context.output['longs']].index.tolist()
    
    #Securties to go short on
    context.shorts = context.output[context.output['shorts']].index.tolist()
    
    
    context.long_weight, context.short_weight = my_compute_weights(context)
    
    

def make_pipeline():
    
    #Universe Q1500US
    base_universe = Q1500US()
    
    #Energy Sector
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    technology_sector = sector.eq(311)
    
    #Make Mask of 1500US and Energy Sector
    base_energy = base_universe & technology_sector
    
    #Dollar Volume (30 days) 
    dollar_volume = AverageDollarVolume(window_length=30)
    
    #Get top 5% in Average Dollar Volume
    top_dollar_volume = dollar_volume.percentile_between(95,100)
    
    #Combine Filters 
    top_five_base_energy = base_energy & top_dollar_volume
    
    #10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], 
                                  window_length=10,
                                  mask=top_five_base_energy)
    
    #30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], 
                                  window_length=30,
                                  mask=top_five_base_energy)
    
    #Percentile Difference
    percent_difference = (mean_10 - mean_30) / mean_30
    
    #List of stocks to short
    shorts = percent_difference < 0
    
    #List of stocks to long
    longs = percent_difference > 0
    
    #Final filter and mask for shorts and longs
    final_securities = (shorts | longs)
    
    #Return the pipeline
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'percent_difference':percent_difference
    },screen=final_securities)

# Backtesting Results
---
*From this it's quite clear to see that focusing on a single industry may be okay for the short run, but for the long run, the index ETF (SPY) will come out ahead*

![](Images/Pipeline_Technology_Sector.png)